## Data Loading and Initial Setup

This cell imports required libraries and loads the cleaned Airbnb dataset. We use:
- pandas for data manipulation
- scikit-learn for machine learning models and preprocessing
- numpy for numerical operations
- matplotlib for visualization

In [ ]:
# Load the cleaned dataset
import pandas as pd

df = pd.read_csv("../ML-Exam/data/cleaned_airbnb_data.csv")

# Show the first few rows and check column names
df.head()


## Initial Random Forest Model

This cell builds and evaluates a Random Forest model to predict Airbnb prices:
1. Encodes categorical features using one-hot encoding
2. Splits data into training and test sets
3. Trains a Random Forest model
4. Evaluates model performance using MSE and R² metrics

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Encode categorical features
df_encoded = pd.get_dummies(df, columns=['room_type', 'City'], drop_first=True)

# Step 2: Define features (X) and target (y)
X = df_encoded.drop(columns=['realSum', 'ID'])
y = df_encoded['realSum']

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Step 5: Predict and evaluate
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2


**Interpretation:**

R² = 0.605 means the model explains about 60.5% of the variance in Airbnb prices.

## Price Distribution Analysis

This cell examines the distribution of prices in the dataset to understand the range and potential outliers.

In [ ]:
df['realSum'].describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99])

## Improved Model with Outlier Removal

This cell creates an improved model by:
1. Removing price outliers above the 99th percentile
2. Retraining the Random Forest model
3. Evaluating the new model's performance

In [ ]:
from sklearn.model_selection import GridSearchCV

# Filter out listings above the 99th percentile
threshold = df['realSum'].quantile(0.99)
df_filtered = df[df['realSum'] < threshold]

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [None, 10],
}


# Create the model
rf = RandomForestRegressor(random_state=42)

# Setup GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    scoring='r2',
    verbose=1
)

# Fit to training data
grid_search.fit(X_train, y_train)

mse, r2


## Save Filtered Dataset

This cell saves the filtered dataset (without outliers) to a CSV file for future use.

In [9]:
filtered_path = "../ML-Exam/data/cleaned_airbnb_data.csv"
df_filtered.to_csv(filtered_path, index=False)